In [1]:
# Cell 1: Setup and Imports

import sys
import os
import pandas as pd


# --- Thêm đường dẫn project vào hệ thống ---
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)


# Cấu hình để Pytorch hoạt động tốt với một số môi trường
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
curr_dir = os.getcwd()
path = os.path.join(curr_dir, "..")

from src.models_lib.model_factory import ModelFactory



/Users/quockhoile/Desktop/DaiHoc/KT/Project/AI-Predict/financial-prediction-system/services/ai-services/.venv/lib/python3.13/site-packages/pytorch_forecasting/models/base/_base_model.py:28: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
from src.data.loader.csv_loader import CSVLoader
from src.data.loader.data_loader_service import DataLoaderService
from src.features.feature_engineer import FeatureEngineer

repository = CSVLoader(path + '/data/raw/day')
data_loader_service = DataLoaderService(repository)

data = data_loader_service.load_data()

fe = FeatureEngineer(lags=[1,7], emas=[10,20], add_volatility=True, add_rsi=True, add_datetime=True)

data_fe = fe.transform(data)

data_fe.head()



,open,high,low,close,volume,symbol,sentiment_score,close_lag_1,close_lag_7,ema_10,ema_20,volatility_10,rsi_14,day_of_week,day_sin,day_cos
timestamp,,,,,,,,,,,,,,,,
2025-01-14,94536.11,97371.00,94346.22,96560.86,27846.61753,BTCUSDT,0.0,94536.10,96954.61,95533.913940,95580.345255,2714.238922,54.182724,1,0.781831,0.623490
2025-01-15,96560.85,100681.94,96500.00,100497.35,30509.99179,BTCUSDT,0.0,96560.86,95060.61,96436.356860,96048.631421,2989.433771,65.051060,2,0.974928,-0.222521
2025-01-16,100497.35,100866.66,97335.13,99987.30,27832.85317,BTCUSDT,0.0,100497.35,92552.49,97081.982885,96423.742715,2537.736889,62.966884,3,0.433884,-0.900969
2025-01-17,99987.30,105865.22,99950.77,104077.48,39171.85292,BTCUSDT,0.0,99987.30,94726.11,98353.891452,97152.670075,3608.518880,70.992879,4,-0.433884,-0.900969
2025-01-18,104077.47,104988.88,102277.55,104556.23,24307.82998,BTCUSDT,0.0,104077.48,94599.99,99481.589370,97857.771020,4306.797050,71.764243,5,-0.974928,-0.222521


In [3]:
from src.models_lib.model_factory import ModelFactory
data_fe.reset_index(drop=False, inplace=True)
model = ModelFactory.get_model("TimeXer", data=data_fe, pred=36, seq=6, path=path + "/models")

model.train()

print(model.evaluate())


NameError: name 'data_fe' is not defined

In [2]:
new_data = pd.read_csv(path + "/data/processed/new_data.csv", parse_dates=["timestamp"])

new_model = ModelFactory.get_model("TimeXer", data=new_data, pred=36, seq=6, path=path + "/models", model_path=path + "/models/days/timexer-36-6.ckpt")

print(new_model.predict(new_data))

/Users/quockhoile/Desktop/DaiHoc/KT/Project/AI-Predict/financial-prediction-system/services/ai-services/.venv/lib/python3.13/site-packages/lightning/pytorch/utilities/parsing.py:210: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
/Users/quockhoile/Desktop/DaiHoc/KT/Project/AI-Predict/financial-prediction-system/services/ai-services/.venv/lib/python3.13/site-packages/lightning/pytorch/utilities/parsing.py:210: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.
/Users/quockhoile/Desktop/DaiHoc/KT/Project/AI-Predict/financial-prediction-system/services/ai-services/.venv/lib/python3.13/site-packages/pytorch_forecasting/models/timexer/_timexer.py:195: UserWarning: In the input sequence, the context_length (36) is not a mu

   timestamp          close
0 2025-05-08  101836.718750
1 2025-05-09   87140.281250
2 2025-05-10   76236.039062
3 2025-05-11   79928.312500
4 2025-05-12   79049.781250
5 2025-05-13   75830.531250


/Users/quockhoile/Desktop/DaiHoc/KT/Project/AI-Predict/financial-prediction-system/services/ai-services/.venv/lib/python3.13/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
/Users/quockhoile/Desktop/DaiHoc/KT/Project/AI-Predict/financial-prediction-system/services/ai-services/.venv/lib/python3.13/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:433: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=9` in the `DataLoader`

In [4]:
import dotenv
from src.data.fecther_factory import FetcherFactory
from src.features.feature_engineer import FeatureEngineer


dotenv.load_dotenv()
api_key = os.getenv("API-Key")
api_secret = os.getenv("Secret-Key")
data_fetcher = FetcherFactory.create_data_fetcher("binance", api_key=api_key, api_secret=api_secret)
btc_data = data_fetcher.fetch_data(
        symbol="BTCUSDT",
        interval="1d", # Lấy khung ngày cho nhanh
        start_str="2025-01-01" # Lấy dữ liệu 1 tháng gần đây
    )
feature = FeatureEngineer(lags=[1,7], emas=[10,20], add_volatility=True, add_rsi=True, add_datetime=True)
btc_data = feature.transform(btc_data)
api_key = os.getenv("TimeGPT-API-Key")
timeGPT = ModelFactory.get_model("TimeGPT", api_key=api_key, seq=6)

timeGPT.predict(btc_data)

/Users/quockhoile/Desktop/DaiHoc/KT/Project/AI-Predict/financial-prediction-system/services/ai-services/src/utils/standardizer.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['timestamp'] = pd.to_datetime(df['timestamp'], unit="ms")
INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Inferred freq: D
/Users/quockhoile/Desktop/DaiHoc/KT/Project/AI-Predict/financial-prediction-system/services/ai-services/.venv/lib/python3.13/site-packages/nixtla/nixtla_client.py:380: UserWarning: `df` contains the following exogenous features: ['open', 'high', 'low', 'volume', 'sentiment_score', 'close_lag_1', 'close_lag_7', 'ema_10', 'ema_20', 'volatility_10', 'rsi_14', 'day_of_week', 'day_sin', 'day_cos'], but `X_df` was not provided 

,symbol,timestamp,close
0,COIN,2025-09-05,110490.09
1,COIN,2025-09-06,110421.98
2,COIN,2025-09-07,110458.67
3,COIN,2025-09-08,110567.94
4,COIN,2025-09-09,110943.69
5,COIN,2025-09-10,111331.69
